# LEE Analyzer notebook

This notebook takes in the output directory of the jobs and convert it into a more flat pandas dataframe. 
Data from the root files will be partially processed to fields that are convenient to plot.
The resulting dataframe will be pickled.

## Imports & Constants

In [1]:
from __future__ import division
from __future__ import print_function

import math
import time
import os
import sys
import glob
import numpy as np
import pandas as pd
from root_pandas import read_root
from helpfunction import sciNot

In [2]:
pd.options.display.max_columns = 999
min_root_size = 20000  # Skip root files smaller than x bytes

vtx_activity_cut = 5  # how many objects start withing 5 cm of the vertex?
z_dead_start = 675
z_dead_end = z_dead_start + 100

In [3]:
# Flat columns we want to copy from the original dataframe:

flat_columns_truth = [
    'nu_pdg',
    'nu_E',
    'true_vx_sce',
    'true_vy_sce',
    'true_vz_sce',
    'distance',
    'ccnc',
    'qsqr',
    'theta',
    'true_1eX_signal',
    'true_nu_fiducial',
    'lepton_E',
    'lepton_theta',
    'true_vx',
    'true_vy',
    'true_vz',
    ]

flat_columns_reco = [
    'event',
    'subrun',
    'run',
    'category',
    'vx',
    'vy',
    'vz',
    'bnbweight',
    'passed',
    'candidate_pdg',
    'numu_cuts',
    'track_bdt_precut',
    'n_showers',
    'n_tracks',
    #'flash_time_max',
    'flash_PE_max',
    'chargecenter_x',
    'chargecenter_y',
    'chargecenter_z',
    'total_spacepoint_containment',
    'fiducial',
    ]

vec_columns_shower = [
    'shower_open_angle',
    'shower_length',
    'shower_start_x',
    'shower_start_y',
    'shower_start_z',
    'shower_dir_x',
    'shower_dir_y',
    'shower_dir_z',
    'shower_pca',
    'shower_maxangle',
    'shower_vtxdistance',
    'shower_daughter',
    'shower_is_daughter',
    'shower_fidvol_ratio',
    'shower_spacepoint_dqdx_ratio',
    'shower_dedx_hits_w',
    'shower_dedx_w',
    'shower_dedx_best_w',
    'shower_energy_w',
    'shower_hitsratio_w',
    'shower_hits_w',
    'shower_theta',
    'shower_phi',
    'shower_energy_product',
    'shower_pitch2',
    'shower_ncluster'
    ]

vec_columns_track = [  
    'track_start_x',
    'track_start_y',
    'track_start_z',
    'track_end_x',
    'track_end_y',
    'track_end_z',
    'track_dir_x',
    'track_dir_y',
    'track_dir_z',
    'track_pca',
    'predict_em',
    'predict_mu',
    'predict_cos',
    'predict_pi',
    'predict_p',
    'track_res_mean',
    'track_res_std',
    'track_maxangle',
    'track_vtxdistance',
    'track_daughter',
    'track_is_daughter',
    'track_spacepoint_dqdx_ratio',
    'track_containment',
    'track_dedx_hits_w',
    'track_dedx_w',
    'track_dedx_best_w',
    'track_energy_w',
    'track_hitsratio_w',
    'track_hits_w',
    'track_theta',
    'track_len',
    'track_phi',
    ]

vec_columns_truth = [
    'true_shower_pdg',
    'true_shower_x_sce',
    'true_shower_y_sce',
    'true_shower_z_sce',
    'true_shower_depE',
    'shower_cle',
    'matched_showers',
    'matched_showers_energy',
    'track_cle',
    'matched_tracks',
    'matched_tracks_energy',
    'nu_daughters_pdg',
    'nu_daughters_E',
    ]

# Columns to use for main frame

columns_data = flat_columns_reco + vec_columns_shower + vec_columns_track
columns_mc = columns_data + flat_columns_truth + vec_columns_truth

# Columns to use for track/shower frame

columns_shower_mc = vec_columns_shower + ['shower_cle', 'matched_showers', 'matched_showers_energy']
columns_track_mc = vec_columns_track + ['track_cle', 'matched_tracks', 'matched_tracks_energy']

columns_flat = [
    'bnbweight',
    'noexpand:1<(n_showers+n_tracks)',
    'fiducial',
    'track_bdt_precut',
    'n_showers',
    'n_tracks',
    'event',
    'subrun',
    'run',
    'candidate_pdg',
    'numu_cuts',
    'category',
    "flash_passed",
    'flash_PE_max',
    'vx',
    'vy',
    'vz',
    'passed',
    'candidate_pdg',
    ]

## Select Input Files

In [4]:
def getFileList(directories):
    directory_in = '/home/wouter/Templates/July/'
    filelist = []
    for directory in directories:
        filelist += glob.glob(directory_in+directory+"/*.root")
    print(len(filelist), 'valid ROOT files collected.')
    return filelist

## Functions

### Main Function: loadData

In [5]:
# list,   Input files.
# bool,   Apply the true signal and save nonpassed events.
# bool,   If this is data, save less stuff
# bool,   Split output in 10 dataframes.
# int,    Maximum number of files to loop over.
# string, Name of the final picle file.
def loadData(  
    filelist,
    signal_sample,
    data,
    split_output,
    maxf=1,
    outputname='output',
    ):

    # Create output directory:
    if not os.path.isdir('../Input/' + outputname):
        print ("Output directory did not exist, creating it.")
        os.system("mkdir "+'../Input/' + outputname)
    else:
        print ("Output directory existed already")
                         
    chunks = []  # list of small dataframes with all info
    chunks_tr = []
    chunks_sh = []
    chunks_nonpassed = []  # list of small dataframes for failed event bookkeeping

    # We store the number of events/wieghted after each selection step
    entries = 0
    entries_weight = 0
    entries_signal = 0
    entries_signal_weight = 0
    flash_time_weight = 0
    flash_50_weight = 0
    flash_passed = 0
    flash_passed_weight = 0
    passed = 0
    passed_weight = 0
    pdg12_passed = 0
    pdg12_passed_weight = 0
    fidvol = 0
    fidvol_weight = 0
    cat2 = 0
    non_passed = 0
    non_passed_weight = 0

    total_pot = 0
    chuncks_pot = 0  # total POT of the sample

    columns_load = columns_data
    columns_track = vec_columns_track
    columns_shower = vec_columns_shower
    
    if not data:
        global columns_flat
        columns_track = columns_track_mc
        columns_shower = columns_shower_mc
        columns_load = columns_mc
        columns_flat += ['true_1eX_signal', 'lepton_theta', 'lepton_E', 'true_vz']

    nfiles = len(filelist)
    if maxf < nfiles:
        nfiles = maxf

    print('Start to load entries from', nfiles, 'files.\n')
    start_time = time.time()

    progress = 0
    progress_pickle = 0
    
    for (i_f, fname) in enumerate(filelist[:nfiles]):
        try:

            # Store the POT of the sample

            df_pot = read_root(fname, 'wouterNueCC/pot')
            temp_pot = df_pot['pot'].sum()
            chuncks_pot += temp_pot
            total_pot += temp_pot

            # Write this dataframe to a txtfile.

            df_pot[['run', 'subrun']].to_csv('../Input/' + outputname
                    + '/run_subrun.txt', header=None, index=None,
                    sep=' ', mode='a')
            dftemp = read_root(fname, 'wouterNueCC/pandoratree',
                               columns=columns_load)
            
            if data or outputname == 'intime':
                dftemp['bnbweight'] = 1

            entries += len(dftemp.index)
            entries_weight += dftemp['bnbweight'].sum()
            # Track/Shower frames

            df_tr = read_root(fname, 'wouterNueCC/pandoratree',
                              columns=columns_track + columns_flat,
                              flatten=columns_track)
                              
            df_sh = read_root(fname, 'wouterNueCC/pandoratree',
                              columns=columns_shower + columns_flat,
                              flatten=columns_shower)
        except (BaseException, e):

            print('Tree corrupt?', fname, '\n', str(e))
            continue

        str_eval_unresponsive_z = 'unresponsive_z = ~( @z_dead_start < true_vz < @z_dead_end)'
        
        str_eval_unresponsive_reco_z = 'unresponsive_reco_z = ~( @z_dead_start < vz < @z_dead_end)'
        dftemp.eval(str_eval_unresponsive_reco_z, inplace=True)

        if signal_sample:
            dftemp = dftemp.query('true_1eX_signal==1')

            dftemp.eval(str_eval_unresponsive_z, inplace=True)
            dftemp = dftemp.query('unresponsive_z==1')
            
            entries_signal += len(dftemp.index)
            entries_signal_weight += dftemp['bnbweight'].sum()

            # Store some basic things about events that did not pass the selection! 
            # (but passed the truth selection)

            str_query = 'n_showers<1 or candidate_pdg!=12 or fiducial==0 or unresponsive_reco_z==0'

            dftemp_nonpassed = dftemp.query(str_query, inplace=False)[flat_columns_reco + flat_columns_truth]

            chunks_nonpassed.append(dftemp_nonpassed)
            non_passed += len(dftemp_nonpassed.index)
            non_passed_weight += dftemp_nonpassed['bnbweight'].sum()

            df_tr.eval(str_eval_unresponsive_z, inplace=True)
            df_sh.eval(str_eval_unresponsive_z, inplace=True)

            str_query = 'true_1eX_signal==1 and unresponsive_z==1'
            df_tr.query(str_query, inplace=True)
            df_sh.query(str_query, inplace=True)
        
        dftemp.query('flash_PE_max>0', inplace=True)
        flash_time_weight += dftemp['bnbweight'].sum()
        
        dftemp.query('flash_PE_max>50', inplace=True)
        flash_50_weight += dftemp['bnbweight'].sum()
        
        dftemp.query('passed==1', inplace=True)
        flash_passed += len(dftemp.index)
        flash_passed_weight += dftemp['bnbweight'].sum()
        
        dftemp.query('n_showers>0', inplace=True)
        passed += len(dftemp.index)
        passed_weight += dftemp['bnbweight'].sum()

        dftemp.query('candidate_pdg==12', inplace=True)
        pdg12_passed += len(dftemp.index)
        pdg12_passed_weight += dftemp['bnbweight'].sum()

        dftemp.query('fiducial==1 & unresponsive_reco_z==1', inplace=True)
        fidvol += len(dftemp.index)
        fidvol_weight += dftemp['bnbweight'].sum()

        cat2 += dftemp.query('category==2')['bnbweight'].sum()

        str_query = 'candidate_pdg==12 & fiducial==1 & n_showers>0 & flash_PE_max>50 & passed==1'
        df_tr.query(str_query, inplace=True)
        df_tr.rename(columns={'1<(n_showers+n_tracks)': 'vtx_activity'}, inplace=True)
        df_sh.query(str_query, inplace=True)
        df_sh.rename(columns={'1<(n_showers+n_tracks)': 'vtx_activity'}, inplace=True)

        # if no events passed, stop here

        if len(dftemp.index) == 0:
            continue

        # Add a feature:

        dftemp['vtx_activity_nr'] = dftemp.apply(lambda x: \
                sum(x['shower_vtxdistance'] < vtx_activity_cut) \
                + sum(x['track_vtxdistance'] < vtx_activity_cut),
                axis=1)

        chunks.append(dftemp)
        chunks_tr.append(df_tr)
        chunks_sh.append(df_sh)

        if (i_f + 1) % math.ceil(nfiles / 10) == 0:
            print('Progress:', (progress + 1) * 10, '%.')
            progress += 1
        
        if split_output:  
            if (i_f + 1) % math.ceil(nfiles / split_output ) == 0:
                print('Concatenating output dataframes')
                print('POT in this chunk:', str(chuncks_pot), 'POT.')
                df = pd.concat(chunks, ignore_index=True, copy=False)
                df.to_pickle('../Input/' + outputname + '/'
                             + outputname + '_' + str(progress_pickle)
                             + '.pckl')
                chunks = []
                chuncks_pot = 0

                df = pd.concat(chunks_sh, ignore_index=True, copy=False)
                df.to_pickle('../Input/' + outputname + '/'
                             + outputname + '_shower_' + str(progress_pickle)
                             + '.pckl')
                chunks_sh = []
                df = pd.concat(chunks_tr, ignore_index=True, copy=False)
                df.to_pickle('../Input/' + outputname + '/'
                             + outputname + '_track_' + str(progress_pickle)
                             + '.pckl')
                chunks_tr = []
                progress_pickle+=1

    end_time = time.time()

    if len(chunks) > 0:
        if split_output:
            progress_pickle==0
            
        print('Concatenating last frame in case of failure, check for double'
              )
        print('POT in this chunk:', str(chuncks_pot), 'POT.')
        df = pd.concat(chunks, ignore_index=True, copy=False)
        df.to_pickle('../Input/' + outputname + '/' + outputname + '_' + str(progress_pickle) + '.pckl')
        chunks = []
        df = pd.concat(chunks_sh, ignore_index=True, copy=False)
        df.to_pickle('../Input/' + outputname + '/' + outputname + '_shower_' + str(progress_pickle) + '.pckl')
        chunks_sh = []
        df = pd.concat(chunks_tr, ignore_index=True, copy=False)
        df.to_pickle('../Input/' + outputname + '/' + outputname + '_track_' + str(progress_pickle) + '.pckl')
        chunks_tr = []

    print('\nSummary:')
    print(
        entries,
        'entries were loaded from',
        nfiles,
        'files, corresponding to',
        str(total_pot),
        'POT.',
        )
    if signal_sample:
        print(round(entries_signal), 'events are 1eX signal in fidvol.')
        print(round(entries_signal_weight), 'weighted signal entries to start with.')
        if round(non_passed + fidvol) != round(entries_signal):
            print(
                  'ERROR: the passing (',fidvol,
                  ') and non-passing (',non_passed,
                  ') events did not sum up correctly to',entries_signal,'!'
                 )
    
    print(round(entries_weight), ' weighted entries to start with.')
    print(round(flash_time_weight), ' events have a flash.')
    print(round(flash_50_weight), ' events have a 50PE flash.')
    print(round(flash_passed_weight), ' events pass the optical precuts.')
    print(round(passed_weight), 'events have a shower.')
    print(round(pdg12_passed_weight), 'events pass the category 12.')
    print(round(fidvol_weight), 'events are in the fiducial volume.')
    print(round(cat2), 'events are category electron neutrino.')

    print('\nLoading took ', sciNot(end_time - start_time), ' seconds.')

    if signal_sample:
        df_nonpassed = pd.concat(chunks_nonpassed, ignore_index=True, copy=False)
        df_nonpassed.to_pickle('../Input/' + outputname + '/' + outputname + '_nonpassed.pckl')

    end2_time = time.time()
    print('Pickling took ', sciNot(end2_time - end_time), ' seconds.')
    print('Done!')

## Load dataframe and save to Pickle

In [6]:
# Load all optical selection frames! 
beam_on = {
    "filelist": ["data_bnb_a_0s0t", "data_bnb_b_0s0t"],
    "signal_sample": False,
    "data": True,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "beam_on"
}

beam_off = {
    "filelist": ["data_bnbext_a_0s0t"],
    "signal_sample": False,
    "data": True,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "beam_off"
}

nu = {
    "filelist": ["bnb_nu_cosmic_0s0t_dev"],
    "signal_sample": False,
    "data": False,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "nu"
}

nue = {
    "filelist": ["bnb_nue_cosmic_0s0t_dev"],
    "signal_sample": True,
    "data": False,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "nue"
}

nu_lightbug = {
    "filelist": ["bnb_nu_cosmic_lightbug"],
    "signal_sample": False,
    "data": False,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "nu_lightbug"
}

nu_induced = {
    "filelist": ["bnb_nu_cosmic_inducedcharge"],
    "signal_sample": False,
    "data": False,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "nu_induced"
}

nu_tune3 = {
    "filelist": ["bnb_nu_cosmic_tune3_0s0t_dev"],
    "signal_sample": False,
    "data": False,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "nu_tune3"
}

nu_overlaid = {
    "filelist": ["bnb_nu_cosmic_overlaid_0s0t_dev"],
    "signal_sample": False,
    "data": True,
    "split_output": 0,
    "maxf" : 1000,
    "outputname" : "nu_overlaid"
}


# add overlaid, cv, lightbug, tune3, (induced charge)
#samples_dict = [nue, beam_on, beam_off, nu, nu_lightbug, nu_induced ,nu_tune3, nu_overlaid]
samples_dict = [beam_off, nue]

for s in samples_dict:
    filelist = getFileList(s["filelist"])
    print(s["outputname"])
    loadData( 
    filelist,
    signal_sample = s["signal_sample"],
    data = s["data"],
    split_output = s["split_output"],
    maxf = s["maxf"],
    outputname = s["outputname"],
    )
    print("\n\n")

741 valid ROOT files collected.
beam_off
Output directory did not exist, creating it.
Start to load entries from 741 files.

Progress: 10 %.
Progress: 20 %.
Progress: 30 %.
Progress: 40 %.
Progress: 50 %.
Progress: 60 %.
Progress: 70 %.
Progress: 80 %.
Progress: 90 %.
Concatenating last frame in case of failure, check for double
POT in this chunk: 143234574.854 POT.

Summary:
433388 entries were loaded from 741 files, corresponding to 143234574.854 POT.
433388  weighted entries to start with.
295955  events have a flash.
294118  events have a 50PE flash.
143086  events pass the optical precuts.
59927 events have a shower.
10896 events pass the category 12.
5865 events are in the fiducial volume.
0 events are category electron neutrino.

Loading took  181.1  seconds.
Pickling took  9.7  seconds.
Done!



282 valid ROOT files collected.
nue
Output directory did not exist, creating it.
Start to load entries from 282 files.

Progress: 10 %.
Progress: 20 %.
Progress: 30 %.
Progress: 40 %.
P

In [7]:
# list,   Input files.
# bool,   Apply the true signal and save nonpassed events.
# bool,   If this is data, save less stuff
# bool,   Split output in 10 dataframes.
# int,    Maximum number of files to loop over.
# string, Name of the final picle file.       
loadData( 
    filelist,
    signal_sample=False,
    data=True,
    split_output=0,
    maxf=5000,
    outputname='bnb',
    )

Output directory did not exist, creating it.
Start to load entries from 282 files.

Progress: 10 %.
Progress: 20 %.
Progress: 30 %.
Progress: 40 %.
Progress: 50 %.
Progress: 60 %.
Progress: 70 %.
Progress: 80 %.
Progress: 90 %.
Concatenating last frame in case of failure, check for double
POT in this chunk: 2.41757769545e+22 POT.

Summary:
197400 entries were loaded from 282 files, corresponding to 2.41757769545e+22 POT.
197400  weighted entries to start with.
161695  events have a flash.
158611  events have a 50PE flash.
111006  events pass the optical precuts.
77131 events have a shower.
51656 events pass the category 12.
33944 events are in the fiducial volume.
23218 events are category electron neutrino.

Loading took  89.2  seconds.
Pickling took  23.5  seconds.
Done!


### Done!